In [2]:
import re

In [3]:
def match(word,string):
    match_string = r'\b' + word + r'\b'
    return bool(re.search(match_string, string))

In [4]:
test = match("219933","CT DEN:904T2390BRMEJCDB 219933")
test

True

In [19]:
test = match(".....","1YnPQ-7l6Ag1FdV")
test

True

In [20]:
test = match("1YnPQ-7l6Ag1FdV","1YnPQ-7l6Ag1FdV")
test

True

In [6]:
test = match("1YnPQ-7l6Ag1FdV","AAAA-1YnPQ-7l6Ag1FdV")
test

True

In [7]:
test = match("1YnPQ-7l6Ag1FdV","1YnPQ-7l6Ag1FdV 123123")
test

True

In [24]:
a = "...............*"
b = "904S2390DP50DBC8"
test = match(a,b.replace('credit','').replace(',','')) & (len(a) >=12)
test

True